In [1]:

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import torch.nn as nn

import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers# MNIST image generation using Conditional DCGAN
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
#import matplotlib.pyplot as plt
import os
#import imageio
#from logger import Logger
#from draw_convnet import DrawNet

Using TensorFlow backend.


In [2]:
# Parameters
image_size = 32
label_dim = 10
G_input_dim = 100
G_output_dim = 1
D_input_dim = 1
D_output_dim = 1
num_filters = [512, 256, 128]

learning_rate = 0.0002
betas = (0.5, 0.999)
batch_size = 128
num_epochs = 20
data_dir = '../../Data/MNIST_data/'
save_dir = 'MNIST_cDCGAN_results/'

In [3]:
# MNIST dataset
transform = transforms.Compose([transforms.Scale(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

mnist_data = dsets.MNIST(root=data_dir,
                         train=True,
                         transform=transform,
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data,
                                          batch_size=batch_size,
shuffle=True)

/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [4]:
# For logger
def to_np(x):
    return x.data.cpu().numpy()


def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [5]:
class Generator(torch.nn.Module):
    def __init__(self, z, c):
        super(Generator, self).__init__()
        self.inputs = torch.cat([z, c], 1)
        self.fc1 = torch.nn.Linear(500, 256)
        self.fc2 = torch.nn.Linear(256,128)

    def forward(self, X):
        
        return F.sigmoid(self.fc2(F.relu(self.fc1(self.inputs))))



In [6]:
class Discriminator(torch.nn.Module):
    def __init__(self, X, c):
        super(Discriminator, self).__init__()
        
        self.fc1 = torch.nn.Linear(128, 256)
        self.fc2 = torch.nn.Linear(256,500)
        
    def forward(self, x):
        return F.sigmoid(self.fc2(F.relu(self.fc1(x))))


In [7]:
# Models
G = Generator(z, c)
D = Discriminator(X, c)
G.cuda()
D.cuda()

NameError: name 'z' is not defined

In [8]:
G

NameError: name 'G' is not defined

In [5]:
# Generator model
class Generator(torch.nn.Module):
    def __init__(self, input_dim, label_dim, num_filters, output_dim):
        super(Generator, self).__init__()

        # Hidden layers
        self.hidden_layer1 = torch.nn.Sequential()
        self.hidden_layer2 = torch.nn.Sequential()
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            # Deconvolutional layer
            if i == 0:
                # For input
                input_deconv = torch.nn.ConvTranspose2d(input_dim, int(num_filters[i]/2), kernel_size=4, stride=1, padding=0)
                self.hidden_layer1.add_module('input_deconv', input_deconv)

                # Initializer
                torch.nn.init.normal(input_deconv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant(input_deconv.bias, 0.0)

                # Batch normalization
                self.hidden_layer1.add_module('input_bn', torch.nn.BatchNorm2d(int(num_filters[i]/2)))

                # Activation
                self.hidden_layer1.add_module('input_act', torch.nn.ReLU())

                # For label
                label_deconv = torch.nn.ConvTranspose2d(label_dim, int(num_filters[i]/2), kernel_size=4, stride=1, padding=0)
                self.hidden_layer2.add_module('label_deconv', label_deconv)

                # Initializer
                torch.nn.init.normal(label_deconv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant(label_deconv.bias, 0.0)

                # Batch normalization
                self.hidden_layer2.add_module('label_bn', torch.nn.BatchNorm2d(int(num_filters[i]/2)))

                # Activation
                self.hidden_layer2.add_module('label_act', torch.nn.ReLU())
            else:
                deconv = torch.nn.ConvTranspose2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

                deconv_name = 'deconv' + str(i + 1)
                self.hidden_layer.add_module(deconv_name, deconv)

                # Initializer
                torch.nn.init.normal(deconv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant(deconv.bias, 0.0)

                # Batch normalization
                bn_name = 'bn' + str(i + 1)
                self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

                # Activation
                act_name = 'act' + str(i + 1)
                self.hidden_layer.add_module(act_name, torch.nn.ReLU())

        # Output layer
        self.output_layer = torch.nn.Sequential()
        # Deconvolutional layer
        out = torch.nn.ConvTranspose2d(num_filters[i], output_dim, kernel_size=4, stride=2, padding=1)
        self.output_layer.add_module('out', out)
        # Initializer
        torch.nn.init.normal(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant(out.bias, 0.0)
        # Activation
        self.output_layer.add_module('act', torch.nn.Tanh())

    def forward(self, z, c):
        h1 = self.hidden_layer1(z)
        h2 = self.hidden_layer2(c)
        x = torch.cat([h1, h2], 1)
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out

In [6]:
# Discriminator model
class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, label_dim, num_filters, output_dim):
        super(Discriminator, self).__init__()

        self.hidden_layer1 = torch.nn.Sequential()
        self.hidden_layer2 = torch.nn.Sequential()
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            # Convolutional layer
            if i == 0:
                # For input
                input_conv = torch.nn.Conv2d(input_dim, int(num_filters[i]/2), kernel_size=4, stride=2, padding=1)
                self.hidden_layer1.add_module('input_conv', input_conv)

                # Initializer
                torch.nn.init.normal(input_conv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant(input_conv.bias, 0.0)

                # Activation
                self.hidden_layer1.add_module('input_act', torch.nn.LeakyReLU(0.2))

                # For label
                label_conv = torch.nn.Conv2d(label_dim, int(num_filters[i]/2), kernel_size=4, stride=2, padding=1)
                self.hidden_layer2.add_module('label_conv', label_conv)

                # Initializer
                torch.nn.init.normal(label_conv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant(label_conv.bias, 0.0)

                # Activation
                self.hidden_layer2.add_module('label_act', torch.nn.LeakyReLU(0.2))
            else:
                conv = torch.nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

                conv_name = 'conv' + str(i + 1)
                self.hidden_layer.add_module(conv_name, conv)

                # Initializer
                torch.nn.init.normal(conv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant(conv.bias, 0.0)

                # Batch normalization
                bn_name = 'bn' + str(i + 1)
                self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

                # Activation
                act_name = 'act' + str(i + 1)
                self.hidden_layer.add_module(act_name, torch.nn.LeakyReLU(0.2))

        # Output layer
        self.output_layer = torch.nn.Sequential()
        # Convolutional layer
        out = torch.nn.Conv2d(num_filters[i], output_dim, kernel_size=4, stride=1, padding=0)
        self.output_layer.add_module('out', out)
        # Initializer
        torch.nn.init.normal(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant(out.bias, 0.0)
        # Activation
        self.output_layer.add_module('act', torch.nn.Sigmoid())

    def forward(self, z, c):
        h1 = self.hidden_layer1(z)
        h2 = self.hidden_layer2(c)
        x = torch.cat([h1, h2], 1)
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out

In [7]:
# Models
G = Generator(G_input_dim, label_dim, num_filters, G_output_dim)
D = Discriminator(D_input_dim, label_dim, num_filters[::-1], D_output_dim)
G.cuda()
D.cuda()

Discriminator(
  (hidden_layer1): Sequential(
    (input_conv): Conv2d (1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (input_act): LeakyReLU(0.2)
  )
  (hidden_layer2): Sequential(
    (label_conv): Conv2d (10, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (label_act): LeakyReLU(0.2)
  )
  (hidden_layer): Sequential(
    (conv2): Conv2d (128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (act2): LeakyReLU(0.2)
    (conv3): Conv2d (256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (act3): LeakyReLU(0.2)
  )
  (output_layer): Sequential(
    (out): Conv2d (512, 1, kernel_size=(4, 4), stride=(1, 1))
    (act): Sigmoid()
  )
)

In [8]:
# Loss function
criterion = torch.nn.BCELoss()

# Optimizers
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [9]:
# Training GAN
D_avg_losses = []
G_avg_losses = []

# Fixed noise & label for test
num_test_samples = 10*10

temp_noise = torch.randn(label_dim, G_input_dim)
fixed_noise = temp_noise
fixed_c = torch.zeros(label_dim, 1)
for i in range(9):
    fixed_noise = torch.cat([fixed_noise, temp_noise], 0)
    temp = torch.ones(label_dim, 1) + i
    fixed_c = torch.cat([fixed_c, temp], 0)

fixed_noise = fixed_noise.view(-1, G_input_dim, 1, 1)
fixed_label = torch.zeros(G_input_dim, label_dim)
fixed_label.scatter_(1, fixed_c.type(torch.LongTensor), 1)
fixed_label = fixed_label.view(-1, label_dim, 1, 1)

# label preprocess
onehot = torch.zeros(label_dim, label_dim)
onehot = onehot.scatter_(1, torch.LongTensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).view(label_dim, 1), 1).view(label_dim, label_dim, 1, 1)
fill = torch.zeros([label_dim, label_dim, image_size, image_size])
for i in range(label_dim):
    fill[i, i, :, :] = 1

step = 0
for epoch in range(num_epochs):
    D_losses = []
    G_losses = []

    if epoch == 5 or epoch == 10:
        G_optimizer.param_groups[0]['lr'] /= 2
        D_optimizer.param_groups[0]['lr'] /= 2

    # minibatch training
    for i, (images, labels) in enumerate(data_loader):

        # image data
        mini_batch = images.size()[0]
        x_ = Variable(images.cuda())

        # labels
        y_real_ = Variable(torch.ones(mini_batch).cuda())
        y_fake_ = Variable(torch.zeros(mini_batch).cuda())
        c_fill_ = Variable(fill[labels].cuda())

        # Train discriminator with real data
        D_real_decision = D(x_, c_fill_).squeeze()
        D_real_loss = criterion(D_real_decision, y_real_)

        # Train discriminator with fake data
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())

        c_ = (torch.rand(mini_batch, 1) * label_dim).type(torch.LongTensor).squeeze()
        c_onehot_ = Variable(onehot[c_].cuda())
        gen_image = G(z_, c_onehot_)

        c_fill_ = Variable(fill[c_].cuda())
        D_fake_decision = D(gen_image, c_fill_).squeeze()
        D_fake_loss = criterion(D_fake_decision, y_fake_)

        # Back propagation
        D_loss = D_real_loss + D_fake_loss
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train generator
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())

        c_ = (torch.rand(mini_batch, 1) * label_dim).type(torch.LongTensor).squeeze()
        c_onehot_ = Variable(onehot[c_].cuda())
        gen_image = G(z_, c_onehot_)

        c_fill_ = Variable(fill[c_].cuda())
        D_fake_decision = D(gen_image, c_fill_).squeeze()
        G_loss = criterion(D_fake_decision, y_real_)

        # Back propagation
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # loss values
        D_losses.append(D_loss.data[0])
        G_losses.append(G_loss.data[0])

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, num_epochs, i+1, len(data_loader), D_loss.data[0], G_loss.data[0]))


Epoch [1/20], Step [1/469], D_loss: 1.5130, G_loss: 0.9261
Epoch [1/20], Step [2/469], D_loss: 1.1526, G_loss: 1.1159
Epoch [1/20], Step [3/469], D_loss: 0.9610, G_loss: 1.4179
Epoch [1/20], Step [4/469], D_loss: 0.7311, G_loss: 1.7454
Epoch [1/20], Step [5/469], D_loss: 0.5587, G_loss: 1.9176
Epoch [1/20], Step [6/469], D_loss: 0.4667, G_loss: 2.0736
Epoch [1/20], Step [7/469], D_loss: 0.3914, G_loss: 2.2124
Epoch [1/20], Step [8/469], D_loss: 0.3535, G_loss: 2.3252
Epoch [1/20], Step [9/469], D_loss: 0.3210, G_loss: 2.3514
Epoch [1/20], Step [10/469], D_loss: 0.3380, G_loss: 2.5202
Epoch [1/20], Step [11/469], D_loss: 0.3319, G_loss: 2.6298
Epoch [1/20], Step [12/469], D_loss: 0.3479, G_loss: 2.6643
Epoch [1/20], Step [13/469], D_loss: 0.3478, G_loss: 2.5132
Epoch [1/20], Step [14/469], D_loss: 0.3190, G_loss: 2.6403
Epoch [1/20], Step [15/469], D_loss: 0.3126, G_loss: 2.7393
Epoch [1/20], Step [16/469], D_loss: 0.3230, G_loss: 2.5883
Epoch [1/20], Step [17/469], D_loss: 0.3579, G_lo

Epoch [1/20], Step [139/469], D_loss: 0.6527, G_loss: 2.0705
Epoch [1/20], Step [140/469], D_loss: 0.9181, G_loss: 4.1973
Epoch [1/20], Step [141/469], D_loss: 1.4358, G_loss: 1.9196
Epoch [1/20], Step [142/469], D_loss: 0.7275, G_loss: 1.8221
Epoch [1/20], Step [143/469], D_loss: 0.5463, G_loss: 2.4270
Epoch [1/20], Step [144/469], D_loss: 0.6335, G_loss: 2.5458
Epoch [1/20], Step [145/469], D_loss: 0.8109, G_loss: 2.9006
Epoch [1/20], Step [146/469], D_loss: 1.0720, G_loss: 1.7198
Epoch [1/20], Step [147/469], D_loss: 1.5643, G_loss: 3.0199
Epoch [1/20], Step [148/469], D_loss: 1.1413, G_loss: 1.2504
Epoch [1/20], Step [149/469], D_loss: 1.1046, G_loss: 2.5200
Epoch [1/20], Step [150/469], D_loss: 0.6256, G_loss: 2.2497
Epoch [1/20], Step [151/469], D_loss: 0.5143, G_loss: 2.5601
Epoch [1/20], Step [152/469], D_loss: 0.4369, G_loss: 2.0383
Epoch [1/20], Step [153/469], D_loss: 0.4077, G_loss: 2.9617
Epoch [1/20], Step [154/469], D_loss: 0.5386, G_loss: 2.7508
Epoch [1/20], Step [155/

Epoch [1/20], Step [275/469], D_loss: 0.7651, G_loss: 1.3608
Epoch [1/20], Step [276/469], D_loss: 1.0865, G_loss: 3.3264
Epoch [1/20], Step [277/469], D_loss: 1.0394, G_loss: 2.5143
Epoch [1/20], Step [278/469], D_loss: 0.6884, G_loss: 1.7871
Epoch [1/20], Step [279/469], D_loss: 0.7136, G_loss: 4.0033
Epoch [1/20], Step [280/469], D_loss: 0.5442, G_loss: 2.8660
Epoch [1/20], Step [281/469], D_loss: 0.4034, G_loss: 2.1848
Epoch [1/20], Step [282/469], D_loss: 0.6867, G_loss: 2.5936
Epoch [1/20], Step [283/469], D_loss: 0.6118, G_loss: 2.6851
Epoch [1/20], Step [284/469], D_loss: 0.7310, G_loss: 2.2336
Epoch [1/20], Step [285/469], D_loss: 0.6092, G_loss: 2.0694
Epoch [1/20], Step [286/469], D_loss: 0.5797, G_loss: 1.8527
Epoch [1/20], Step [287/469], D_loss: 0.4490, G_loss: 2.4315
Epoch [1/20], Step [288/469], D_loss: 0.5133, G_loss: 2.7083
Epoch [1/20], Step [289/469], D_loss: 0.5633, G_loss: 1.9233
Epoch [1/20], Step [290/469], D_loss: 0.6729, G_loss: 3.5254
Epoch [1/20], Step [291/

Epoch [1/20], Step [411/469], D_loss: 0.9627, G_loss: 1.5853
Epoch [1/20], Step [412/469], D_loss: 0.8741, G_loss: 2.2782
Epoch [1/20], Step [413/469], D_loss: 0.6246, G_loss: 1.4558
Epoch [1/20], Step [414/469], D_loss: 0.6999, G_loss: 2.4132
Epoch [1/20], Step [415/469], D_loss: 0.7516, G_loss: 2.5062
Epoch [1/20], Step [416/469], D_loss: 0.4951, G_loss: 1.1584
Epoch [1/20], Step [417/469], D_loss: 0.7763, G_loss: 2.4832
Epoch [1/20], Step [418/469], D_loss: 0.5812, G_loss: 2.1022
Epoch [1/20], Step [419/469], D_loss: 0.4927, G_loss: 2.3892
Epoch [1/20], Step [420/469], D_loss: 0.3780, G_loss: 2.5328
Epoch [1/20], Step [421/469], D_loss: 0.4537, G_loss: 1.6798
Epoch [1/20], Step [422/469], D_loss: 0.6251, G_loss: 1.9484
Epoch [1/20], Step [423/469], D_loss: 0.9721, G_loss: 1.4104
Epoch [1/20], Step [424/469], D_loss: 0.6222, G_loss: 2.6152
Epoch [1/20], Step [425/469], D_loss: 0.6798, G_loss: 1.9666
Epoch [1/20], Step [426/469], D_loss: 0.7286, G_loss: 1.6043
Epoch [1/20], Step [427/

Epoch [2/20], Step [78/469], D_loss: 0.8854, G_loss: 1.3770
Epoch [2/20], Step [79/469], D_loss: 0.6862, G_loss: 1.7036
Epoch [2/20], Step [80/469], D_loss: 0.6444, G_loss: 1.9351
Epoch [2/20], Step [81/469], D_loss: 0.4937, G_loss: 2.4515
Epoch [2/20], Step [82/469], D_loss: 0.6104, G_loss: 1.7573
Epoch [2/20], Step [83/469], D_loss: 0.6497, G_loss: 0.8489
Epoch [2/20], Step [84/469], D_loss: 0.7454, G_loss: 2.1238
Epoch [2/20], Step [85/469], D_loss: 0.5901, G_loss: 1.7886
Epoch [2/20], Step [86/469], D_loss: 0.4450, G_loss: 2.2668
Epoch [2/20], Step [87/469], D_loss: 0.5458, G_loss: 2.0557
Epoch [2/20], Step [88/469], D_loss: 0.6213, G_loss: 1.7803
Epoch [2/20], Step [89/469], D_loss: 0.6108, G_loss: 1.7176
Epoch [2/20], Step [90/469], D_loss: 0.8732, G_loss: 1.4877
Epoch [2/20], Step [91/469], D_loss: 0.7865, G_loss: 0.9258
Epoch [2/20], Step [92/469], D_loss: 0.8792, G_loss: 2.9210
Epoch [2/20], Step [93/469], D_loss: 0.5795, G_loss: 2.3640
Epoch [2/20], Step [94/469], D_loss: 0.6

...

Epoch [19/20], Step [337/469], D_loss: 0.4380, G_loss: 2.1786
Epoch [19/20], Step [338/469], D_loss: 0.6279, G_loss: 1.6854
Epoch [19/20], Step [339/469], D_loss: 0.9348, G_loss: 1.4883
Epoch [19/20], Step [340/469], D_loss: 0.5038, G_loss: 2.1345
Epoch [19/20], Step [341/469], D_loss: 0.5930, G_loss: 2.2074
Epoch [19/20], Step [342/469], D_loss: 0.5655, G_loss: 2.1363
Epoch [19/20], Step [343/469], D_loss: 0.6860, G_loss: 1.5526
Epoch [19/20], Step [344/469], D_loss: 1.6769, G_loss: 2.4550
Epoch [19/20], Step [345/469], D_loss: 1.0447, G_loss: 1.6965
Epoch [19/20], Step [346/469], D_loss: 0.5336, G_loss: 1.1818
Epoch [19/20], Step [347/469], D_loss: 0.5747, G_loss: 1.6542
Epoch [19/20], Step [348/469], D_loss: 0.9551, G_loss: 2.8910
Epoch [19/20], Step [349/469], D_loss: 0.9081, G_loss: 1.2006
Epoch [19/20], Step [350/469], D_loss: 0.6570, G_loss: 1.2357
Epoch [19/20], Step [351/469], D_loss: 0.7166, G_loss: 1.9566
Epoch [19/20], Step [352/469], D_loss: 0.3967, G_loss: 2.1365
Epoch [1

Epoch [20/20], Step [2/469], D_loss: 0.4607, G_loss: 1.6236
Epoch [20/20], Step [3/469], D_loss: 0.4583, G_loss: 2.0033
Epoch [20/20], Step [4/469], D_loss: 0.3622, G_loss: 2.2309
Epoch [20/20], Step [5/469], D_loss: 0.7780, G_loss: 1.1354
Epoch [20/20], Step [6/469], D_loss: 0.5460, G_loss: 1.3225
Epoch [20/20], Step [7/469], D_loss: 0.6451, G_loss: 1.8666
Epoch [20/20], Step [8/469], D_loss: 0.9891, G_loss: 2.6310
Epoch [20/20], Step [9/469], D_loss: 0.9606, G_loss: 2.3628
Epoch [20/20], Step [10/469], D_loss: 0.5622, G_loss: 0.9147
Epoch [20/20], Step [11/469], D_loss: 0.9092, G_loss: 2.0853
Epoch [20/20], Step [12/469], D_loss: 0.4779, G_loss: 1.9476
Epoch [20/20], Step [13/469], D_loss: 0.6174, G_loss: 1.0297
Epoch [20/20], Step [14/469], D_loss: 0.4447, G_loss: 1.5605
Epoch [20/20], Step [15/469], D_loss: 0.6124, G_loss: 1.4661
Epoch [20/20], Step [16/469], D_loss: 0.5578, G_loss: 2.0557
Epoch [20/20], Step [17/469], D_loss: 0.4732, G_loss: 2.2080
Epoch [20/20], Step [18/469], D_

Epoch [20/20], Step [136/469], D_loss: 0.7761, G_loss: 3.6578
Epoch [20/20], Step [137/469], D_loss: 0.8200, G_loss: 1.2253
Epoch [20/20], Step [138/469], D_loss: 0.5837, G_loss: 1.4820
Epoch [20/20], Step [139/469], D_loss: 0.7125, G_loss: 2.0395
Epoch [20/20], Step [140/469], D_loss: 0.5929, G_loss: 2.3314
Epoch [20/20], Step [141/469], D_loss: 0.8376, G_loss: 1.7089
Epoch [20/20], Step [142/469], D_loss: 0.3510, G_loss: 1.5343
Epoch [20/20], Step [143/469], D_loss: 0.5255, G_loss: 1.5249
Epoch [20/20], Step [144/469], D_loss: 0.6328, G_loss: 1.2544
Epoch [20/20], Step [145/469], D_loss: 0.5484, G_loss: 1.6951
Epoch [20/20], Step [146/469], D_loss: 0.6651, G_loss: 1.1036
Epoch [20/20], Step [147/469], D_loss: 0.7526, G_loss: 2.0195
Epoch [20/20], Step [148/469], D_loss: 0.4697, G_loss: 1.9136
Epoch [20/20], Step [149/469], D_loss: 0.6601, G_loss: 1.9310
Epoch [20/20], Step [150/469], D_loss: 0.9962, G_loss: 1.3925
Epoch [20/20], Step [151/469], D_loss: 0.7714, G_loss: 1.2744
Epoch [2

Epoch [20/20], Step [270/469], D_loss: 0.8771, G_loss: 1.7824
Epoch [20/20], Step [271/469], D_loss: 0.2323, G_loss: 1.6321
Epoch [20/20], Step [272/469], D_loss: 0.4549, G_loss: 2.5561
Epoch [20/20], Step [273/469], D_loss: 0.4016, G_loss: 2.0861
Epoch [20/20], Step [274/469], D_loss: 0.5904, G_loss: 1.6629
Epoch [20/20], Step [275/469], D_loss: 0.7759, G_loss: 1.0956
Epoch [20/20], Step [276/469], D_loss: 0.5612, G_loss: 1.1002
Epoch [20/20], Step [277/469], D_loss: 0.4644, G_loss: 1.4312
Epoch [20/20], Step [278/469], D_loss: 0.7389, G_loss: 2.6407
Epoch [20/20], Step [279/469], D_loss: 0.6039, G_loss: 2.4662
Epoch [20/20], Step [280/469], D_loss: 0.8881, G_loss: 1.3522
Epoch [20/20], Step [281/469], D_loss: 0.6246, G_loss: 1.2144
Epoch [20/20], Step [282/469], D_loss: 0.4893, G_loss: 0.5181
Epoch [20/20], Step [283/469], D_loss: 0.6262, G_loss: 2.5047
Epoch [20/20], Step [284/469], D_loss: 1.1278, G_loss: 1.3374
Epoch [20/20], Step [285/469], D_loss: 0.5585, G_loss: 1.2601
Epoch [2

Epoch [20/20], Step [404/469], D_loss: 0.4837, G_loss: 1.5985
Epoch [20/20], Step [405/469], D_loss: 0.5392, G_loss: 1.2689
Epoch [20/20], Step [406/469], D_loss: 0.9622, G_loss: 2.7677
Epoch [20/20], Step [407/469], D_loss: 0.3508, G_loss: 2.1013
Epoch [20/20], Step [408/469], D_loss: 0.7858, G_loss: 1.2653
Epoch [20/20], Step [409/469], D_loss: 0.7359, G_loss: 2.0497
Epoch [20/20], Step [410/469], D_loss: 0.4490, G_loss: 1.9227
Epoch [20/20], Step [411/469], D_loss: 0.5081, G_loss: 1.9514
Epoch [20/20], Step [412/469], D_loss: 0.3218, G_loss: 1.2951
Epoch [20/20], Step [413/469], D_loss: 0.6098, G_loss: 0.9694
Epoch [20/20], Step [414/469], D_loss: 0.5985, G_loss: 1.3877
Epoch [20/20], Step [415/469], D_loss: 0.5781, G_loss: 1.8066
Epoch [20/20], Step [416/469], D_loss: 0.4409, G_loss: 1.0862
Epoch [20/20], Step [417/469], D_loss: 0.5335, G_loss: 2.5659
Epoch [20/20], Step [418/469], D_loss: 0.8776, G_loss: 1.2528
Epoch [20/20], Step [419/469], D_loss: 0.5157, G_loss: 1.0334
Epoch [2